# Path Configuration

In [1]:
import os
import sys
import argparse

root = "../"*3
src_path = os.path.join(root, "kyoka")
sample_path = os.path.join(root, "sample")
sys.path.append(root)
sys.path.append(src_path)
sys.path.append(sample_path)

import logging as log
log.basicConfig(format='[%(levelname)s] %(message)s', level=log.DEBUG)

from kyoka.algorithm.td_learning.deep_q_learning import DeepQLearning as DQN

from kyoka.policy.epsilon_greedy_policy import EpsilonGreedyPolicy
from kyoka.finish_rule.watch_iteration_count import WatchIterationCount

from sample.maze.maze_domain import MazeDomain
from sample.maze.maze_dqn_value_function import MazeDQNValueFunction
from sample.maze.maze_helper import MazeHelper
from sample.maze.maze_performance_logger import MazePerformanceLogger
from sample.maze.maze_transformer import MazeTransformer

# Define Const for Performance Test

# Setup Global Item for Performance Test

In [89]:
%matplotlib inline
import seaborn
import matplotlib.pyplot as plt

maze_file_path = lambda maze_type: "../script/%s.txt" % maze_type
transform_file_path = lambda maze_type: "../script/%s_transformed.txt" % maze_type

def gen_callbacks(maze_type, transform_timing):
    callbacks = [MazePerformanceLogger()]
    if maze_type in ["blocking", "shortcut"]:
        transfomer = MazeTransformer()
        transfomer.set_transformation(transform_timing, transform_file_path(maze_type))
        callbacks.append(transfomer)
    return callbacks

def run_performance_test(maze_type, rl_algo, epsilon, test_length, transform_timing):
    watch_iteration = WatchIterationCount(target_count=test_length, log_interval=1000)
    finish_rules = [watch_iteration]
    domain = MazeDomain()
    domain.read_maze(maze_file_path(maze_type))
    value_func = MazeTableValueFunction(domain.get_maze_shape())
    value_func.setUp()
    policy = EpsilonGreedyPolicy(domain, value_func, eps=epsilon)
    callbacks = gen_callbacks(maze_type, transform_timing)
    [rl_algo.set_gpi_callback(callback) for callback in callbacks]
    rl_algo.GPI(domain, policy, value_func, finish_rules)
    return callbacks[0].step_log, callbacks[0].policy_log

def visualize_maze(maze_type):
    domain = MazeDomain()
    domain.read_maze(maze_file_path(maze_type))
    print domain.get_maze_shape()
    print MazeHelper.visualize_maze(domain.maze)
    
def visualize_transform_maze(maze_type, transform_timing):
    domain = MazeDomain()
    domain.read_maze(maze_file_path(maze_type))
    print "maze shape ( height=%d, width=%d )" % domain.get_maze_shape()
    print
    print "Initial maze shape is ..."
    print
    print MazeHelper.visualize_maze(domain.maze)
    print
    print "After %d iteration, maze transforms into ..." % transform_timing
    print
    domain.read_maze(transform_file_path(maze_type))
    print MazeHelper.visualize_maze(domain.maze)

def visualize_step_transition(step_log):
    print "minimum step => %d" % min(step_log)
    plt.plot(step_log, label="step")
    plt.xlabel("GPI iteration")
    plt.ylabel("step")
    plt.show()
    
def visualize_policy_transition(step_log, policy_log, sampling_interval):
    sampled_log = [(item[0]+1, item[1]) for item in enumerate(policy_log) if (item[0]+1)%sampling_interval==0]
    for iteration, log in sampled_log:
        minimum_step = min(step_log[:iteration])
        print "After %d th iteration (minimum step => %d)" % (iteration, minimum_step)
        print log
        print

# DynaMaze

In [59]:
MAZE_TYPE = "dyna"
TRANSFORM_TIMING = 0
visualize_maze(MAZE_TYPE)

(6, 9)
-------XG
--X----X-
S-X----X-
--X------
-----X---
---------


In [ ]:
TEST_LENGTH = 500
EPSILON = 0.3
step_log, policy_log = run_performance_test(
    maze_type=MAZE_TYPE, rl_algo=DQN(N=100, C=100, minibatch_size=32, replay_start_size=50),
    epsilon=EPSILON, test_length=TEST_LENGTH, transform_timing=TRANSFORM_TIMING)
visualize_step_transition(step_log)
visualize_policy_transition(step_log, policy_log, sampling_interval=50)

<img src="./resource/dqn_dyna_maze_step_transition.png" width=600 />

```
minimum step => 14
After 50 th iteration (minimum step => 14)
>>>v>vv-G
-^vv>vv-^
-v-<vvv^^
vv^>>>>>^
>>>>^^v>^
>^^^^>v<^

After 100 th iteration (minimum step => 14)
>>>vvvv-G
>vv>>vv-^
>v-vvvv^^
vv^>>>>>^
>>>^^>>^^
^>>>^>^^^

After 150 th iteration (minimum step => 14)
>>>vvvv-G
^^v>>>v-^
vv-vvvv>^
>v^>>>>>^
>>>>^^^>^
>>>>^>>^^

After 200 th iteration (minimum step => 14)
>>>>>vv-G
>^v>vvv-^
>v->v>v>^
vv^>>>>>^
>>>^^>>^^
^>>>>>>^^

After 250 th iteration (minimum step => 14)
>>>v>vv>G
^vvv>vv>^
vv->>vv>^
vv^>>>>>^
>>>>^^>>^
>>^>>>^^^

After 300 th iteration (minimum step => 14)
>>>v>vv>G
>^vvvvv>^
vv->>vv>^
>v^>>>>>^
>>>>^>^^^
>>>^^>^^^

After 350 th iteration (minimum step => 14)
>>>>>vv>G
^^v>vvv>^
vv-vvvv>^
>v^>>>>>^
>>>^^^>^^
>>>>>>^^^

After 400 th iteration (minimum step => 14)
>>>vvvv>G
>vv>vvv>^
vv->vvv>^
>v^>>>>>^
>>>>^^>>^
>^>^^>^^^

After 450 th iteration (minimum step => 14)
>>>>vvv>G
^vvvvvv>^
vv->>vv>^
>v^>>>>>^
>>>>^>>^^
>^>>^>^^^

After 500 th iteration (minimum step => 14)
>>>v>vv>G
^^vvvvv>^
vv->>>v>^
>v^>>>>>^
>>>^^>>>^
>>>>>>^>^

training time of 500 GPI : 1821.574468
```

# Blocking Maze

In [ ]:
MAZE_TYPE = "blocking"
TRANSFORM_TIMING = 200
visualize_maze(MAZE_TYPE, TRANSFORM_TIMING)

```
maze shape ( height=6, width=9 )

Initial maze shape is ...

--------G
---------
---------
XXXXXXXX-
---------
---S-----

After 200 iteration, maze transforms into ...

--------G
---------
---------
-XXXXXXXX
---------
---S-----
```

In [ ]:
TEST_LENGTH = 500
EPSILON = 0.3
step_log, policy_log = run_performance_test(
    maze_type=MAZE_TYPE, rl_algo=DQN(N=100, C=100, minibatch_size=32, replay_start_size=50),
    epsilon=EPSILON, test_length=TEST_LENGTH, transform_timing=TRANSFORM_TIMING)
visualize_step_transition(step_log)
visualize_policy_transition(step_log, policy_log, sampling_interval=50)

<img src="./resource/dqn_blocking_maze_step_transition.png" />

```
minimum step => 10
After 50 th iteration (minimum step => 10)
v>>^>>>>G
<v^^>^^^^
>v<v>v<>^
-------<^
---->>>v^
----->>v<

After 100 th iteration (minimum step => 10)
v>>^>>>>G
<v^^>^>>^
>v<v>v>>^
-------<^
>>>>>>>>^
-^>>^>^^^

After 150 th iteration (minimum step => 10)
v>>^>>>>G
<v^^>^>>^
>v<v>v>>^
-------<^
>>>>>>>>^
-^^>>>>^^

After 200 th iteration (minimum step => 10)
v>>^>>>>G
<v^^>^>>^
>v<v>v>>^
-------<^
>>>>>>>>^
-^>>>>^>^

After 250 th iteration (minimum step => 16)
->>>>>>>G
^^>^>^^^^
^^v>^^>>^
^>-------
^<<<<<<>-
^<<<^<<^^

After 300 th iteration (minimum step => 16)
>>>>>>>>G
>>^>>^>^^
>^^^^^^^^
^>-------
^<<<<<<>-
^^<^^<<^^

After 350 th iteration (minimum step => 16)
>>>>>>>>G
>>>>^^^^^
^>^>^>^^^
^>-------
^<<<>^<>-
^^^<v^<^^

After 400 th iteration (minimum step => 16)
>>>>>>>>G
>>>>^>^^^
>>^>^^>^<
^>-------
^<<<<>>>-
^<<<>>^^^

After 450 th iteration (minimum step => 16)
>>>>>>>>G
>>>>>^>^^
>^^>^^^^^
^>-------
^<<<<----
^^^<-----

After 500 th iteration (minimum step => 16)
>>>>>>>>G
>>>>>>>^^
>>^^^^^^^
^>-------
^<<<<->>v
^^<<-->>-

training time of 500 GPI : 2458.186745
```

# Shortcut Maze

In [ ]:
MAZE_TYPE = "shortcut"
TRANSFORM_TIMING = 200
visualize_maze(MAZE_TYPE, TRANSFORM_TIMING)

```
maze shape ( height=6, width=9 )

Initial maze shape is ...

--------G
---------
---------
-XXXXXXXX
---------
---S-----

After 200 iteration, maze transforms into ...

--------G
---------
---------
-XXXXXXX-
---------
---S-----
```

In [ ]:
TEST_LENGTH = 500
EPSILON = 0.3
step_log, policy_log = run_performance_test(
    maze_type=MAZE_TYPE, rl_algo=DQN(N=100, C=100, minibatch_size=32, replay_start_size=50),
    epsilon=EPSILON, test_length=TEST_LENGTH, transform_timing=TRANSFORM_TIMING)
visualize_step_transition(step_log)
visualize_policy_transition(step_log, policy_log, sampling_interval=50)

<img src="./resource/dqn_shortcut_maze_step_transition.png" />

```
minimum step => 10
After 50 th iteration (minimum step => 16)
>>>>>>>>G
>>>>>>>^^
>>>>>>>>^
^>------^
^<<v<v>^v
^<>v<v<>-

After 100 th iteration (minimum step => 16)
>>>>>>>>G
>>>>>>^>^
>>>^>^^^^
^>------^
^<<<<<>^<
^^^^<<^^-

After 150 th iteration (minimum step => 16)
>>>>>>>>G
>>>>^^^^^
>>>^^^^^^
^>------^
^<<<<<>^<
^^^<^^^^^

After 200 th iteration (minimum step => 16)
>>>>>>>>G
>>>>>^>^^
>>>^>^^^^
^>------^
^<<<<^>^<
^<<^<^^^^

After 250 th iteration (minimum step => 16)
>>>>>>>>G
>>>>>^>^^
^^>>^^^^^
^>-----<^
^<<<<<>>^
^<^<<^>^^

After 300 th iteration (minimum step => 16)
>>>>>>>>G
>>>^>^>^^
^>^>^^^^^
^>-----<^
^<<<<<>>^
^<^^^^>^^

After 350 th iteration (minimum step => 16)
>>>>>>>>G
>>>>>>^>^
>>^>>^^^^
^>-----<^
^<<<<<>>^
^^<<^^>^^

After 400 th iteration (minimum step => 10)
>>>>>>>>G
>>>>^>^^^
^^^>^>>>^
^>-----<^
^<<<>>>>^
^<<<>^>^^

After 450 th iteration (minimum step => 10)
>>>>>>>>G
>>^^>^>^^
>^>>>^^^^
^>-----<^
^>>>>>>>^
^^>>>>>>^

After 500 th iteration (minimum step => 10)
>>>>>>>>G
>>^^>^>>^
>^>>>^>>^
^>-----<^
^>>>>>>>^
^>^>>^^>^

training time of 500 GPI : 1775.471908(s)
```